In [1]:
#Imports
from deap import creator, tools, base
import numpy as np
import timeit
from stage import fullPrint, getStage
from datetime import datetime, date, timedelta
from sklearn.base import BaseEstimator
import random
import pandas as pd
import yahoo_fin.stock_info as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
start = timeit.default_timer()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [3]:
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
np.random.shuffle(train)
trainSet1, trainSet2, trainSet3, trainSet4, trainSet5 = np.array_split(train,5)
trainSets = [trainSet1, trainSet2, trainSet3, trainSet4, trainSet5]

In [4]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0,0,1,0.1,1,0.995,0.9,0.985])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.05))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.0005))
noises.append(np.random.normal(0, 0.0025))

noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [5]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))


In [6]:
def evalReturn(individual):
    print(individual[0])
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > individual[0][9] and df.loc[date,'RS'] > individual[0][10]:
                    if df.at[date,'close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'ticker'])
            except: 
                pass
        goodSectorDf.at[date,'Sectors'] = set(listOfSector) 
    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf, trainSets)
    if stockStageEstimator.score() <  0.7:
        print(stockStageEstimator.result())
        return 0,
    print(stockStageEstimator.result())
    return stockStageEstimator.result(),

In [7]:
toolbox.register("evaluate", evalReturn)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [8]:
class StockStageEstimator(BaseEstimator):
    def __init__(self, paramList = [0.20, 1.39, 0.35, 0.93, 1.09, 1.22, 0.98, 0.98, 1.38, 0.99, -0.01, 0.97, 0.99, 0.96, 0.98], goodSectorDf = pd.DataFrame, sets = [[]]):
        self.paramList = paramList
        self.goodSectorDf = goodSectorDf
        self.returns = [0,0,0,0,0,0,0,0,0,0]
        self.sets = sets
    
    def evalFit(self, tickers, goodSectorDf):
        transactionFit = pd.read_pickle("transactionTemplate.pkl")
        transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    if df.at[date,'MA30Slope'] > self.paramList[9] and df.loc[date,'RS'] > self.paramList[10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
        for symbol in tickers:
            df = getStage(symbol,self.paramList,goodSectorDf)
            inStage = False
            buyTwice = False
            if df.empty:
                continue
            drift = -1
            for index, element in df.iterrows():
                if drift == -1:
                    drift = index.weekday()
                index = index - timedelta(days= drift)
                index = index.strftime('%Y-%m-%d')
                if element.Stage == "Stage 2" or element.Stage == "Buy":
                    transactionFit.loc[index]['holding'].append((symbol,element.close,0))
                    if buyTwice:
                        transactionFit.loc[index]['holding'].append((symbol,element.close,0))
                    if element.Stage == "Buy" and inStage:
                        transactionFit.loc[index]['holding'].append((symbol,element.close,0))
                        buyTwice = True
                    inStage = True
                    continue
                if element.Stage == "Sell":
                    transactionFit.loc[index]['holding'].append((symbol,element.close,-1))
                    if buyTwice:
                        transactionFit.loc[index]['holding'].append((symbol,element.close,-1))
                    inStage = False
                    buyTwice = False
        total = 100
        prevIndex = ''
        first = True
        transactionFitCopy = transactionFit
        transactionFitCopy['total'] = 0
        for index, element in transactionFitCopy.iterrows():
            index = index.strftime('%Y-%m-%d')
            if len(element.holding) == 0:
                transactionFitCopy['total'][index] = total
                continue
            if first:
                first = False
                transactionFitCopy['total'][index] = total
            else:
                prevData = transactionFitCopy.loc[prevIndex].holding
                if len(prevData):
                    total = 0
                    removeList = []
                    close = 0
                    for i in range(len(prevData)):
                        for j in range(len(element.holding)):
                            if element.holding[j][0] == prevData[i][0]:
                                close = element.holding[j][1]
                                if element.holding[j][2] == -1:
                                    removeList.append(element.holding[j])
                                break
                        total += close*prevData[i][2]
                    transactionFitCopy['total'][index] = total
                    for delete in removeList:
                        if delete in element.holding:
                            element.holding.remove(delete)
                else:
                    transactionFitCopy['total'][index] = transactionFitCopy.loc[prevIndex].total
            if len(element.holding):
                allocation = total/len(element.holding)
            for i in range(len(element.holding)):
                element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
            prevIndex = index
        stockHolding  = 0
        for i in transactionFitCopy.iterrows():
            stockHolding += len(i[1]['holding'])
        if stockHolding/(transactionFitCopy.shape[0])<8 or transactionFitCopy.iloc[-1]['total']<=100:
            return -1
        return transactionFitCopy.iloc[-1]['total']

    def fit(self):
        for i in range(10):
            if i%2 == 0:
                setsCombined = np.concatenate((self.sets[(i+1)%5],self.sets[(i+2)%5],self.sets[(i+3)%5],self.sets[(i+4)%5]))
                self.returns[i] = self.evalFit(setsCombined, self.goodSectorDf)
            else:
                self.returns[i] = self.evalFit(self.sets[i], self.goodSectorDf)
            if self.returns[i] == -1:
                return -1
        return 0
    
    def score(self):
        if self.fit() == -1:
            return -1
        return r2_score([self.returns[i*2] for i in range(5)],[self.returns[i*2+1] for i in range(5)])
    
    def result(self):
        return np.average(self.returns)
        

In [9]:
def main():
    pop = toolbox.population(n=4)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        if ind.fitness.values[0]<=100:
            del ind.fitness.values
    badInd = [ind for ind in pop if not ind.fitness.valid]
    while len(badInd)!=0:
        for ind in badInd:
            index = pop.index(ind)
            temp = toolbox.individual()
            pop[index] = temp
            del pop[index].fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, badInd)
        for ind, fit in zip(badInd, fitnesses):
            ind.fitness.values = fit
            if ind.fitness.values[0]<=100:
                del ind.fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
    
        
    # Begin the evolution
    for g in range(2):
        print("-- Generation %i --" % g)
        resultFile = open("resultML1.txt","a")
        resultFile.write("-- Generation %i --" % g+"\n")
        resultFile.close()
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [10]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
main()

[0.3285922738837642, 0.9831865854865053, -0.009889132721897592, 0.9468884605379647, 1.070379409479322, 1.1472760389027965, 0.8452066346422369, 1.037116411119654, 1.2811731732403404, 0.9981175193740753, 0.1779124943219769, 0.9614166701376545, 0.9977965430106555, 0.8688304456981317, 0.9873857365772561]
1851.235661754807
[0.21752586041083877, 1.1880814088386062, 0.3062570744767915, 0.9093819919588398, 1.3430287912236882, 1.0645210533587341, 0.8183375485193336, 1.035008966028344, 1.3619136964428626, 0.9876622506749922, 0.1143264575687595, 1.0704666643010636, 1.0007914348273785, 0.9385105342663598, 0.9846888901308297]
-0.1
[0.6424693546060491, 0.8336931733663203, -0.09582667242799062, 1.0483938436609814, 1.179526201224029, 1.0946472760114474, 0.8471470045068052, 1.08662268953076, 1.2220504223176967, 0.9906438754391937, 0.12514322131532385, 1.003579770816047, 0.9939275529002235, 0.8921943011292247, 0.9777499835752317]
-0.1
[0.26393383086127575, 1.4189117986429465, 0.36888088262907004, 0.9591